### Imports and function definitions

In [18]:
#This directory contains all the data needed for the module. It should be in the same directory as the notebook
dataFolder = "data"


import sys, os, shutil, errno, string, urllib


sys.path.append(( os.path.abspath( os.path.join(os.path.realpath("__file__"),  os.pardir, os.pardir) )))
import uvmodlib.v1 as uvmod

### USES Conf Templates

In [109]:
GRACETemplate = """mesh
{  
    data  sphere ./modules/$folderName/sphere.obj
    cullRadius $cr
    glslVersion 330
    
    propertyCollection 
    {        
        __objectName__
        {           
            vec3f p1min  -0.02 -1.12  -5.05
            vec3f p1max   0.05  0.56   3.02
            vec3f p2min  -3.98 -0.33 -15.82
            vec3f p2max   8.28  0.33  32.39 
            vec1f cmin -56.142 #-38.815 #
            vec1f cmax  34.589 #19.123 #
            vec1f t0 2003.015
            vec1f tf 2016.536
            vec1f simUseTime  2003.015
            bool simBindRealtime  false
            bool simBindLimits  true
        }
    }

    pass 
    {
        useDataObject sphere
        shader
        {
            type defaultMeshShader
            {
                vertexShader   ./modules/$folderName/GRACEmap.vs
                fragmentShader ./modules/$folderName/GRACEmap.fs
                
                StateManagerVar __objectName__.simUseTime simUseTime
                StateManagerVar __objectName__.simBindRealtime simBindRealtime
                StateManagerVar __objectName__.simBindLimits simBindLimits
                StateManagerVar __objectName__.p1min p1min
                StateManagerVar __objectName__.p2min p2min
                StateManagerVar __objectName__.p1max p1max
                StateManagerVar __objectName__.p2max p2max
                StateManagerVar __objectName__.cmin cmin
                StateManagerVar __objectName__.cmax cmax
                StateManagerVar __objectName__.t0 t0
                StateManagerVar __objectName__.tf tf

                stateManagerVar __objectName__.Alpha        uv_alpha

                texture fit1 ./modules/$folderName/GRACEfit_1.png
                {
                }
                texture fit2 ./modules/$folderName/GRACEfit_2.png
                {
                }
                texture coast ./modules/$folderName/CoastLines.png
                {
                }  
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }
    

    
    
}"""

### GRACEmap Class

In [110]:
class GRACEmap():
    def __init__(self, object):
        self.object = object
        uvmod.Utility.ensurerelativepathexsists("sphere.obj",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("GRACEmap.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("GRACEmap.fs",dataFolder)        
        self.cr = 1000
    def generatemod(self):
        self.object.setgeometry(self.object.name+"Mesh.usesconf")
        return self.object.generatemod()
    def generatefiles(self, absOutDir, relOutDir):
        fileName = self.object.name+"Mesh.usesconf"
        s = string.Template(GRACETemplate)
        f = open(absOutDir+"\\"+fileName, 'w')
        if f:
            f.write(s.substitute(folderName = relOutDir,
                                 cr = self.cr))
            f.close()
        uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, absOutDir)

### Object Instantiation

In [111]:
GRACE = GRACEmap(uvmod.OrbitalObject())
generator = uvmod.Generator()
scene = uvmod.Scene()
modinfo = uvmod.ModuleInformation()

### Specify Settings and generate the module

In [113]:
GRACE.object.setcameraradius(10)
GRACE.object.setcoord("Earth")
GRACE.object.setname("GRACE")
GRACE.object.setguiname("/KavliLecture/Schrag/GRACE map")
GRACE.object.settargetradius(2000)
GRACE.object.setscalefactor(660.0)
GRACE.object.showatstartup(False)
GRACE.object.custom = "bin 9999"
GRACE.object.custom += """
    rotation
    {
        spice
        {
            frame		IAU_EARTH 
            reference	ECLIPJ2000
        }
    }"""
GRACE.cr = 10000



modinfo.setname("GRACE Earth map")
modinfo.setauthor("Aaron Geller<sup>1</sup> and Daniel Schrag<sup>2</sup><br />(1)Adler Planetarium,<br />(2)Harvard University")
modinfo.cleardependencies()
modinfo.adddependency("Earth")
modinfo.setdesc("A module to show the data from the GRACE satellite. <br /> \
                Using the GSFC solution. The aata comes from here: https://neptune.gsfc.nasa.gov/gngphys/index.php?section=413")
#modinfo.setthumbnail("data/thumb.png")
modinfo.setversion("1.0")

generator.generate("GRACE",[],[GRACE],modinfo)
uvmod.Utility.senduvcommand(GRACE.object.name+".reload;")

## To reload

In [23]:
from config import Settings

In [117]:
uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, Settings.uvcustommodulelocation+'\\'+GRACE.object.name)
uvmod.Utility.senduvcommand(GRACE.object.name+".reload;")